In [33]:
import numpy as np
from keras.datasets import mnist

In [34]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [35]:
x_train.shape

(60000, 28, 28)

In [36]:
images = x_train[:1000].reshape(1000, 28*28)
labels = y_train[0:1000].reshape(1000,1)

In [37]:
mean_x = np.mean(images)
std = np.sum(images**2)/(images.shape[0]-1)

In [38]:
images = (images - mean_x)/std
images

array([[-0.00310983, -0.00310983, -0.00310983, ..., -0.00310983,
        -0.00310983, -0.00310983],
       [-0.00310983, -0.00310983, -0.00310983, ..., -0.00310983,
        -0.00310983, -0.00310983],
       [-0.00310983, -0.00310983, -0.00310983, ..., -0.00310983,
        -0.00310983, -0.00310983],
       ...,
       [-0.00310983, -0.00310983, -0.00310983, ..., -0.00310983,
        -0.00310983, -0.00310983],
       [-0.00310983, -0.00310983, -0.00310983, ..., -0.00310983,
        -0.00310983, -0.00310983],
       [-0.00310983, -0.00310983, -0.00310983, ..., -0.00310983,
        -0.00310983, -0.00310983]])

In [39]:
from neural_network import NeuralNetwork

In [56]:
count_of_features = images.shape[1]
count_classes = 10
batch_size = 200
learing_rate = 0.001

In [57]:
nn = NeuralNetwork()

In [58]:
nn.CreateLayerRelu((count_of_features, 1000))
nn.CreateLayerRelu((1000, 500))
nn.CreateLayerLinear((500, count_classes))

In [62]:
epochs = 10
for it in range(epochs):
    batch_indexes = np.random.randint(0, images.shape[0], batch_size)
    batch_X = images[batch_indexes]
    batch_y = labels[batch_indexes]
    output = nn.forward(batch_X)
    prediction = nn.softmax(output)
    loss = nn.CrossEntropyLoss(prediction, batch_y)
    nn.backward(learing_rate)
    print(loss)

22.634909618401572
13.813140518849655
7.9989119107853295
5.264154721438031
5.0216904766404085
3.655061454447534
3.2841105326954994
3.5816382804242863
3.147557574474125
1.8707036019439738
